# **Playing Fetch with Spot**

The goal of this script is to add pretrained models to spot which can detect objects.
This should allow a user to play fetch with spot.

## Collect Images

Our goal is to collect images of the dog in as many environments as possible.

In [1]:
# Activate environment (PowerShell): spot_env\Scripts\activate

# Install OpenCV's python bindings in your virtualenv:
!py.exe -3 -m pip install opencv-python==4.5.* -q

# Install numpy
!pip install numpy -q

# cd ./Object detection/fetch

# Running our code to capture images
!py.exe -3 capture_images.py 192.168.80.3 --image-source right_fisheye_image --folder dogtoy/images

## Label Images

To label our images, we’ll use an open source off-the-shelf labeling program. 

In [4]:
!py.exe -3 -m pip install labelme -q
!labelme

In the labelImg display, open the image directory with File > Open Dir

The main keys we'll use are:
- d     Next image
- a     Previous image
- w     Create a box
- del	Delete the selected box

Use d to get to an image that has the dog toy in it. Press w then click-and-drag to make a box on the toy. Enter dogtoy into the pop-up and press OK.

Tip: If the box refuses to be a rectangle, press Ctrl to disable that mode.

## Training the model

#### Installations

Install NVIDIA drivers, CUDA, and cuDNN

- <font color='Yellow'>What is NVIDIA?</font>

NVIDIA is known for developing integrated circuits, which are used in everything from electronic game consoles to personal computers (PCs). The company is a leading manufacturer of high-end graphics processing units (GPUs).

A driver is a software that enables the operating system to communicate with hardware or a device. Thus, for an NVIDIA graphics driver, it allows the OS to communicate with the video card, ensuring that it functions properly.

You need to have a real, physical NVIDIA graphics card in order to install NVIDIA drivers.

Link: https://helpx.adobe.com/x-productkb/multi/drivers-video-win-nvidia.html

- <font color='Yellow'>What is CUDA?</font>

NVIDIA provides something called the Compute Unified Device Architecture (CUDA), which is crucial for supporting the various deep learning applications. 

CUDA is a framework that allows people with a Nvidia Graphics Card to use GPU acceleration when it comes to deep learning. 

Link: https://developer.nvidia.com/cuda-downloads

- <font color='Yellow'>What is cuDNN?</font>

The cuDNN library is a library optimized for CUDA containing GPU implementations. Think of cuDNN as a library for Deep Learning using CUDA and CUDA as a way to talk to the GPU.

Link: https://docs.nvidia.com/deeplearning/cudnn/install-guide/index.html


Install TensorFlow

Activate virtual environment (copy/paste to terminal):

spot_env\Scripts\activate

In [ ]:
# Tensorflow likes to install a non-GUI version of OpenCV, which will cause us problems later
# We can safely uninstall it because we already installed OpenCV.

!py.exe -3 -m pip install --upgrade pip
!py.exe -3 -m pip install tensorflow-gpu==2.3.1 tensorflow==2.3.1 tensorboard==2.3.0 tf-models-official==2.3.0 pycocotools lvis
!py.exe -3 -m pip uninstall opencv-python-headless

In [6]:
# Test TensorFlow and CUDA installation (you should have at least 1 GPU as a result):
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


Install TensorFlow Object Detection API

In [ ]:
# Download our package with the precompiled files and save it in the fetch folder
unzip models-with-protos.zip

# Install the object detection API:
# cd models-with-protos/research
!py.exe -3 -m pip install .

## Prepare Training Data

First convert JSON files to XML files

In [ ]:
!py.exe -3 -m pip install json2xml
!py.exe -3 -m pip install dict2xml

import json2xml
import dict2xml

In [7]:
def json2xml(json_obj, line_padding=""):
    
    result_list = list()

    json_obj_type = type(json_obj)

    if json_obj_type is list:
        for sub_elem in json_obj:
            result_list.append(json2xml(sub_elem, line_padding))

        return "\n".join(result_list)

    if json_obj_type is dict:
        for tag_name in json_obj:
            sub_obj = json_obj[tag_name]
            result_list.append("%s<%s>" % (line_padding, tag_name))
            result_list.append(json2xml(sub_obj, "\t" + line_padding))
            result_list.append("%s</%s>" % (line_padding, tag_name))

        return "\n".join(result_list)

    return "%s%s" % (line_padding, json_obj)

In [27]:
import json

path = 'C:/Users/AD655YE/OneDrive - EY\Desktop/Spot/Object detection/fetch/dogtoy/annotations/'
image = 'back_fisheye_image_0000'

with open(f"{path}{image}.json") as input_var:
    j = load(input_var)

x = json2xml(j)
#print(x)

with open(f"{path}{image}.xml", mode='w') as out:
    out.write(x)


Partition the data into training and test sets. Download the script split_dataset.py and put it in the ~/fetch folder. Run the script to copy the XML label files into the train and test folders, splitting them up randomly.

In [ ]:
# cd ~/fetch
!python split_dataset.py --labels-dir dogtoy/annotations/ --output-dir dogtoy/annotations/ --ratio 0.9

Create a label map by creating a file called label_map.pbtxt and put it in dogtoy/annotations. 

Convert labels to .record format. Download the script generate_tfrecord.py and place it in the fetch directory.

In [ ]:
# run for training set
!py.exe -3 generate_tfrecord.py --xml_dir dogtoy/annotations/train --image_dir dogtoy/images --labels_path dogtoy/annotations/label_map.pbtxt --output_path dogtoy/annotations/train.record

# run again for test set
!py.exe -3 generate_tfrecord.py --xml_dir dogtoy/annotations/test --image_dir dogtoy/images --labels_path dogtoy/annotations/label_map.pbtxt --output_path dogtoy/annotations/test.record

## Train the Model

Download Transfer Learning Model:

Make a pre-trained-models folder: 

mkdir dogtoy/pre-trained-models

We'll use the SSD ResNet50 V1 FPN 640x640 model. Download it into the ~/fetch/dogtoy/pre-trained-models folder. The TensorFlow Model Zoo has lots of other models you could use.

Extract it:

cd dogtoy/pre-trained-models
tar -zxvf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

Set up model parameters:

Make a folder for our new model: 

cd ~/fetch/dogtoy
mkdir -p models/my_ssd_resnet50_v1_fpn

Copy the pre-trained model parameters:

cp pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config models/my_ssd_resnet50_v1_fpn/